In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import re, string
import emoji
from sklearn import preprocessing


Đọc data từ file lấy ra các cột chuẩn

In [7]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
df=pd.DataFrame()
df['clean_text']=X_tr[0].astype(str)
df['sentiment']=Y_tr[0]
df_test=pd.DataFrame()
df_test['clean_text']=X_t[0].astype(str)
df_test['sentiment']=Y_t[0]
text_len = []
for text in df.clean_text:
    tweet_len = len(text.split())
    text_len.append(tweet_len)
df['text_len'] = text_len
text_len_test = []
for text in df_test.clean_text:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)
df_test['text_len'] = text_len_test
df=df[df['text_len']>4]
df_test=df_test[df_test['text_len']>4]
print(df.shape)
print(df_test.shape)

(24086, 3)
(3099, 3)


In [9]:
df.to_csv('Twitter_2/data_train.csv',index=False)
df_test.to_csv('Twitter_2/data_test.csv',index=False)

In [8]:
df=pd.read_csv('Twitter_3/Corona_NLP_train.csv',encoding='ISO-8859-1')
df_test=pd.read_csv('Twitter_3/Corona_NLP_test.csv',encoding='ISO-8859-1')
df.drop_duplicates(subset='OriginalTweet',inplace=True)
df_test.drop_duplicates(subset='OriginalTweet',inplace=True)
df = df[['OriginalTweet','Sentiment']]
df_test = df_test[['OriginalTweet','Sentiment']]
valid_sentiments = ["Extremely Positive", "Positive", "Neutral", "Extremely Negative", "Negative"]

df = df.dropna()
df_test = df_test.dropna()
df = df[df['Sentiment'].isin(valid_sentiments)]      
df_test = df_test[df_test['Sentiment'].isin(valid_sentiments)]
df.to_csv('Twitter_3/Corona_NLP_train.csv',index=False)
df_test.to_csv('Twitter_3/Corona_NLP_test.csv',index=False)
print(df.shape)
print(df_test.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'Twitter_3/Corona_NLP_train.csv'

Các hamfd lằm sạch văn bản

In [3]:
def strip_emoji(text):
    return emoji.replace_emoji(text, "")
def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() 
    text = re.sub(r'(?:\@|https?\://)\S+', "", text)
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) 

    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) 
    return new_tweet2
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text):
    return re.sub("\s\s+" , " ", text)


<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:26: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\w'
<>:26: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Temp\ipykernel_17784\1412012263.py:12: SyntaxWarning: invalid escape sequence '\w'
  new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet))
C:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Temp\ipykernel_17784\1412012263.py:26: SyntaxWarning: invalid escape sequence '\s'
  return re.sub("\s\s+" , " ", text)


Thwucj hiện làm sạch văn bản tính toán ddookj dài văn bản và loại bỏ các văn bản có độ dài bé hơn 4 lưu văn bản vào file

In [4]:
texts_new=[]
for t in df.OriginalTweet:
    texts_new.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))
texts_new_test=[]
for t in df_test.OriginalTweet:
    texts_new_test.append(remove_mult_spaces(filter_chars(clean_hashtags(strip_all_entities(strip_emoji(t))))))
df_tr=pd.DataFrame(texts_new,columns=['clean_text'])
df_t=pd.DataFrame(texts_new_test,columns=['clean_text'])
text_len = []
for text in df_tr.clean_text:
    tweet_len = len(text.split())
    text_len.append(tweet_len)
df_tr['text_len'] = text_len
text_len_test = []
for text in df_t.clean_text:
    tweet_len = len(text.split())
    text_len_test.append(tweet_len)
df_t['text_len'] = text_len_test
label=[]
for x in df.Sentiment:
    if x=='Negative' or x=='Extremely Negative':
       label.append(0)
    elif x=='Neutral':
        label.append(1)
    elif x=='Positive' or x=='Extremely Positive':
        label.append(2)

df_tr['Sentiment']=label
label_test=[]
for x in df_test.Sentiment:
    if x=='Negative' or x=='Extremely Negative':
       label_test.append(0)
    elif x=='Neutral':
        label_test.append(1)
    elif x=='Positive' or x=='Extremely Positive':
        label_test.append(2)
df_t['Sentiment']=label_test
df_tr = df_tr[df_tr['text_len']>4]
df_t=df_t[df_t['text_len']>4]
df_tr.to_csv('Twitter_3/data_train.csv',index=False)
df_t.to_csv('Twitter_3/data_test.csv',index=False)

C:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Temp\ipykernel_17784\1412012263.py:12: SyntaxWarning: invalid escape sequence '\w'
  new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet))
C:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Temp\ipykernel_17784\1412012263.py:26: SyntaxWarning: invalid escape sequence '\s'
  return re.sub("\s\s+" , " ", text)


TypeError: object of type 'float' has no len()